In [1]:
import numpy as np
import math
import matplotlib.pyplot
## Calcul de IC pour les quantiles
from scipy.stats import binom
from math import ceil
def IC_quantile(quantiles, niveau):
    n = len(quantiles)
    quantiles = np.sort(quantiles)
    index_ = ceil(n * niveau) - 1
    niveau_inf = ceil(n * (niveau - 1.96 * np.sqrt(niveau * (1-niveau) / n))) - 1
    niveau_sup = ceil(n * (niveau + 1.96 * np.sqrt(niveau * (1-niveau) / n))) - 1
    q = quantiles[index_]
    print("Quantile au niveau {} = {}, avec intervalle de confiance = {}, {}".format(niveau, 
                                                                                     q, 
                                                                                     quantiles[niveau_inf], 
                                                                                     quantiles[niveau_sup]))

In [2]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#### On définit nu, la loi des incréments
# Ancien : plus lent
saut_1_ancien = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2_ancien = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3

# Nouveau : beaucoup plus vite
value_1 = np.array([-1, 1])
value_2 = np.array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
saut_1 = lambda x : value_1[np.random.randint(low=2, size=x)]
saut_2 = lambda x : value_2[np.random.randint(low=12, size=x)]

# Les paramètres
P0 = 35
T = 4*60*60
lamb = 1/300

In [3]:
P0 = 35
T = 4*60*60
lamb = 1/300
N=100
alpha = -0.875

value_2 = np.array([1, 1, 1, 2, 2, 3])
saut_1 = lambda x: np.ones(shape=x) #correspond à m=1
saut_2 = lambda x: value_2[np.random.randint(low=6, size=x)] #correspond à m=3

In [4]:
def part_interaction(N,saut,P0,M,alpha,mu):
    #mu: paramètre des poids G_p
    # Tirages de M x N v.a. suivant la loi "saut" et de MxN v.a. delta_n représentant le changement de signe du saut par rapport au signe du saut précédent
    # seront utilisees pour construire les chaînes de Markov
    xi= saut((M, N))   #N: horizon temporel (nombre de sauts)
    delta = -1+2*np.random.binomial(1,(1+alpha)/2,size=(M,N-1))

    ###############################################################################
    ## En ponderant les trajectoires hautes : Estimation via G(X)=e^{alpha X_p}
    ###############################################################################

    # P stocke TOUTES LES VALEURS pour chacune des M chaines, dans l'optique du calcul de G_p 
    # J_hat stocke la dernière valeur du signe de saut (saut vers le bas ou vers le haut) pour chacune des M chaînes
    P = np.zeros((M,N))

    J_hat = -1+2*np.random.binomial(1,0.5,size=M) #le premier saut est vers le bas ou vers le haut avec la même probabilité

    # On initialise avec la valeur du prix après le premier saut
    P[:,0] = P0+J_hat*xi[:,0]
        
    facteurDroite = 1 #produit des moyennes des pondérations à chaque itération

    for k in range(N-1):
                
        weights = np.exp(-mu*P[:,k])
        
        facteurDroite *= np.mean(weights)
        
        weights /= np.sum(weights)

        I = np.random.choice(np.arange(M),size=M,replace=True,p=weights)
        
        #sélection
        P = P[I,:]
        J_hat=J_hat[I]

        #mutation

        J_hat=J_hat*delta[:,k]
        P[:,k+1] = P[:,k] +xi[:,k+1]*J_hat

    # pour ajuster mu; il faut que le prix final soit souvent négatif 
    # mais pas qu'il parte trop loin dans les négatifs
    print("prix final=",P[:10,-1])
    tabFacteursGauche = np.exp(mu*np.sum(P[:,:-1],axis=1)) #tableau des 1/(\prod_{k=0}^{N-2} G_k)
    proba_negatif = np.sum(tabFacteursGauche[np.min(P,axis=1)<0])/M * facteurDroite
    return proba_negatif

In [14]:
%%time
M= int(1e7)
P0= 35
N=100
mu = 0.08

#proba_negatif = Q3_proba_negatif(N,saut_2,P0,M,alpha)
proba_negatif = part_interaction(N,saut_2,P0,M,alpha,mu)
print(proba_negatif)
#Pour P0=10 et m=3 et M=10^5 et N=100, donne 0.2262 avec MC naif
#Donne 0.2331 avec méthode particules en interaction (mu=0.03/P0)
#pour P0=35, particules en interaction donne 5.027933935719282e-05

prix final= [-19.  12. -14. -26.  -6.  -3.  -9.   9.  -5.  -5.]
4.1720376497474846e-05
CPU times: user 20min 47s, sys: 2min 34s, total: 23min 21s
Wall time: 24min 22s


In [14]:
%%time
M= int(1e7)
P0= 35
N=100
mu = 0.4/P0 #ajusté empiriquement
proba_negatif = part_interaction(N,saut_1,P0,M,alpha,mu)
print(proba_negatif)

prix final= [33. 21. 29. 29. 33. 33. 37. 33. 25. 33.]
0.0
CPU times: user 21min, sys: 3min 27s, total: 24min 27s
Wall time: 32min 20s


In [5]:
#Monte-Carlo naïf - Méthode 1

def Q3_proba_negatif(N,saut,P0,M,alpha): 
    # Sauts
    increments = saut((M, N))
    # Signes
    signes = -1 + 2 * np.random.binomial(1, (1 + alpha) / 2, size=(M, N)) # 1 : rester le même signe
    signes[:, 0] = -1 + np.random.binomial(1, 0.5, size=M) * 2 # Début : -1 ou 1 de proba 1/2
    signes = np.cumprod(signes, axis=1)
    # Prix -> P0 + sauts*signes
    prix = P0 + np.cumsum(increments * signes, axis=1)
    # La probabilité d'avoir un prix < 0
    proba_negatif = np.mean(prix.min(axis=1) < 0)
    
    return proba_negatif, prix[:, -1] # prix[:, -1] : prix final


In [12]:
%%time
M= int(1e7)
P0= 35
N=100
proba_negatif, prix = Q3_proba_negatif(N,saut_2,P0,M,alpha)
print(proba_negatif)
prix.sort()
print(prix[int(M * 1e-5) - 1], prix[int(M * 1e-4) - 1], prix[int(M - M * 1e-4)], prix[int(M - M * 1e-5)])

4.58e-05
-2 3 67 72
CPU times: user 30.1 s, sys: 8.66 s, total: 38.7 s
Wall time: 41.8 s


In [13]:
%%time
M= int(1e7)
P0= 35
N=100
proba_negatif, prix = Q3_proba_negatif(N,saut_1,P0,M,alpha)
print(proba_negatif)
prix.sort()
print(prix[int(M * 1e-5) - 1], prix[int(M * 1e-4) - 1], prix[int(M - M * 1e-4)], prix[int(M - M * 1e-5)])

0.0
23.0 25.0 45.0 47.0
CPU times: user 21.8 s, sys: 7.49 s, total: 29.3 s
Wall time: 31.5 s


### Estimation de quantiles du prix final après N sauts

In [5]:
#Quantile simple 


def Quantile_naif_2(seuil,N,saut,P0,M,alpha):

    liste_P_t = np.zeros(M)
    for i in range(M):
        signes = -1+2*np.random.binomial(1,(1+alpha)/2,size=N)
        signes[0] = -1+np.random.binomial(1,0.5)*2
        signes = np.cumprod(signes)
        increments = saut(N)
        
        liste_P_t[i] = np.sum(increments*signes)+P0 
    return liste_P_t

In [6]:
%%time
seuil = 1e-2
M=int(1e7)
quantiles = Quantile_naif_2(seuil,N,saut_1,P0,M,alpha)
IC_quantile(quantiles, 1e-4)
IC_quantile(quantiles, 1e-5)
#print("Quantiles de niveau {} : ({} +/ {:08.7f}, {} +/ {})".format(alpha,estimateur_quantiles[0],erreur[0],estimateur_quantiles[1],erreur[1]))

Quantile au niveau 0.0001 = 25.0, avec intervalle de confiance = 25.0, 25.0
Quantile au niveau 1e-05 = 23.0, avec intervalle de confiance = 23.0, 23.0
CPU times: user 3min 13s, sys: 219 ms, total: 3min 13s
Wall time: 3min 13s


In [7]:
%%time
seuil = 1e-2
M=int(1e7)
quantiles = Quantile_naif_2(seuil,N,saut_2,P0,M,alpha)
IC_quantile(quantiles, 1e-4)
IC_quantile(quantiles, 1e-5)

Quantile au niveau 0.0001 = 3.0, avec intervalle de confiance = 3.0, 3.0
Quantile au niveau 1e-05 = -2.0, avec intervalle de confiance = -2.0, -2.0
CPU times: user 5min 13s, sys: 3.33 s, total: 5min 16s
Wall time: 5min 13s


### Limite macroscopique

#### prix negatif

In [4]:
#Monte-Carlo 
from numba import jit

@jit(nopython=True)
def proba_negatif(N, signes, increments, P0):
    ruines = 0
    for i in range(len(N) - 1):
        somme = P0
        ruine = 0
        signe = signes[N[i]]
        for j in range(N[i], N[i+1]):
            somme += signe * increments[j]
            if somme < 0:
                ruine = 1
                break
            signe *= signes[j+1]
        ruines += ruine
    return ruines
    

def Q3_limite_macro(lamb, T,saut,P0,M,alpha):
    # Sauts
    N = np.random.poisson(lamb*T, size=M + 1)
    print(N.mean())
    N[0] = 0
    N = N.cumsum()
    
    increments = saut(N[-1] + 1)
    # Signes
    signes = -1 + 2 * np.random.binomial(1, (1 + alpha) / 2, size=N[-1] + 1) # 1 : rester le même signe
    signes[N[:-1]] = -1 + np.random.binomial(1, 0.5, size=M) * 2 # Début : -1 ou 1 de proba 1/2
    
    return proba_negatif(N, signes, increments, P0) / M
    

In [16]:
%%time
T = 8*250*60*60
lamb = 1/300
M = int(1e5)
P0=1000

proba = Q3_limite_macro(lamb,T,saut_2,P0,M,alpha)
print(proba)
print(1.96 * np.sqrt(proba * (1 - proba) / M))

24000.387786122137
0.0
0.0
CPU times: user 49.4 s, sys: 35 s, total: 1min 24s
Wall time: 5min 51s


In [6]:
proba = Q3_limite_macro(lamb,T,saut_1,P0,M,alpha)
print(proba)
print(1.96 * np.sqrt(proba * (1 - proba) / M))

0.0
0.0


#### Quantile

In [28]:
from numba import jit

@jit(nopython=True)
def quantiles_macro(N, signes, increments, P0):
    quantiles = np.zeros(len(N) - 1)
    for i in range(len(N) - 1):
        somme = P0
        signe = signes[N[i]]
        for j in range(N[i], N[i+1]):
            somme += signe * increments[j]
            signe *= signes[j+1]
        quantiles[i] = somme
    return quantiles
    

def Q3_limite_macro_quantile(lamb, T,saut,P0,M,alpha):
    # Sauts
    N = np.random.poisson(lamb*T, size=M + 1)
    N[0] = 0
    N = N.cumsum()
    
    increments = saut(N[-1] + 1)
    # Signes
    signes = -1 + 2 * np.random.binomial(1, (1 + alpha) / 2, size=N[-1] + 1) # 1 : rester le même signe
    signes[N[:-1]] = -1 + np.random.binomial(1, 0.5, size=M) * 2 # Début : -1 ou 1 de proba 1/2
    
    return quantiles_macro(N, signes, increments, P0)

In [29]:
proba = Q3_limite_macro_quantile(lamb,T,saut_1,P0,M,alpha)
print(proba[int(M*1e-6)], proba[int(M - M*1e-6)])

[1319.  946.  990.  934.  752.  929.  909.  823. 1275.  630. 1122. 1294.
 1193. 1007.  830.  873. 1265. 1104. 1165.  976. 1162.  752. 1189.  936.
  974.  904. 1240.  792.  755. 1234. 1011. 1134.  841.  846. 1232.  815.
 1284. 1000.  984.  697.  933. 1198.  953.  726. 1309.  807. 1101. 1258.
  972. 1185. 1276.  968.  802. 1267.  868.  549. 1013.  886. 1016.  977.
 1285.  772. 1261. 1083.  815. 1071. 1115.  978.  505.  965.  966.  952.
  765. 1118.  922.  864.  698.  958. 1016.  936.  929.  598. 1099.  928.
 1081.  754.  807. 1138.  752. 1212.  977. 1149.  938. 1183. 1124. 1064.
 1288.  775.  995. 1317.]


In [ ]:
proba = Q3_limite_macro_quantile(lamb,T,saut_2,P0,M,alpha)
print(proba[int(M*1e-6)], proba[int(M - M*1e-6)])